# Bitácora de exploración
## Collective Intelligence - Making Recommendations

### Filtros colaborativos

Se desarrollará un sistema de recomendación basado coincidencias de un usuario con otros.

Un algorítmo de *filtrado colaborativo* usualmente funciona buscando en un grupo grande de personas y encontrando un conjunto mas pequeño con gustos similares a los de cierto usuario.

En general, filtrado colaborativo es el proceso de filtrado de información o patrones usando técnicas que involucran la colaboración entre múltiples agentes, puntos de vista, fuentes de datos, etc.

### Recolectando preferencias

Lo primero que se tiene que considerar es una representación que nos permita hablar de **personas** y sus **gustos** sobre determinados **ítems**. En este caso las personas serán críticos de películas, los gustos se representarán como las calificaciones que le asignan a películas, las cuales serán los ítems.

La representación con la que se trabajará será la de diccionarios anidados, si `d[a,b]` es un diccionario que relaciona `a` con `b`, la estructura que utilizaremos tiene la forma:

```
d[Nombre de persona, d[Nombre de película, Calificación]
```

Tanto el nombre del crítico como el nombre de la película son cadenas de caracteres; la calificación se representa como un valor numérico entre 1 y 5

In [46]:
# Un diccionario de críticos de cine con sus calificaciones de un
# conjunto pequeño de películas
critics = {
    "Lisa Rose" : {
        "Lady in the Water"  : 2.5,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 3.0,
        "Superman Returns"   : 3.5,
        "You, Me and Dupree" : 2.5,
        "The Night Listener" : 3.0
    },
    "Gene Seymour" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 1.5,
        "Superman Returns"   : 5.0,
        "The Night Listener" : 3.0,
        "You, Me and Dupree" : 3.5
    },
    "Michael Phillips" : {
        "Lady in the Water"  : 2.5,
        "Snakes on a Plane"  : 3.0,
        "Superman Returns"   : 3.5,
        "The Night Listener" : 4.0
    },
    "Claudia Puig" : {
        "You, Me and Dupree" : 2.5,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 3.0,
        "Superman Returns"   : 4.0,
        "The Night Listener" : 4.5
    },
    "Mick LaSalle" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 4.0,
        "Just My Luck"       : 2.0,
        "Superman Returns"   : 3.0,
        "The Night Listener" : 3.0,
        "You, Me and Dupree" : 2.0
    },
    "Jack Matthews" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 4.0,
        "You, Me and Dupree" : 3.5,
        "Superman Returns"   : 5.0,
        "The Night Listener" : 3.0
    },
    "Toby" : {
        "Snakes on a Plane"  : 4.5,
        "You, Me and Dupree" : 1.0,
        "Superman Returns"   : 4.0
    },
    "Lolo" : {
        "Lili" : 1,
        "Lala" : 2
    },
    "Coco" : {
        "Cici" : 5.0,
        "Caca" : 2.4
    },
    "Bobo" : {
        "Lili" : 3,
        "Lala" : 4
    },
    "Gerardo" : {
        "Lady in the Water"  : 3.1,
        "You, Me and Dupree" : 3.0,
        "Superman Returns"   : 2.9
    },
    "Christian" : {
        "Lady in the Water"  : 2.9,
        "You, Me and Dupree" : 3.0,
        "Superman Returns"   : 3.1
    },
    "Luque" : {
        "Lady in the Water"  : 1.0
    },
    "Moises" : {
        "Lady in the Water"  : 3.5,
        "Superman Returns"   : 4.1
    },
}

### Encontrando usuarios similares

Para determinar que tan similar son los gustos de dos personas se pueden utilizar varias métricas, las cuales llamamos *puntuaciones de similitud*

En este ejercicio se implementarán dos: la *distancia Euclideana* y la *correlación de Pearson*

#### Puntuación de la distancia Euclideana

Considerando $n$ películas calificadas por una cantidad de críticos, podemos encontrar que tan similar son dos personas si tomamos un espacio $n$-dimensional en donde un punto es un crítico y sus coordenadas se determinan por la calificación que le asignó dicho crítico a las películas.

La distancia Euclideana en este caso será calculada entre dos críticos considerando las películas que calificaron en común.

$$f(c^{(a)}, c^{(b)}) = \sqrt{(c^{(a)}_1 - c^{(b)}_1)^2 + \dots (c^{(a)}_n - c^{(b)}_n)^2}$$

donde $c^{(r)}_{i}$ es la calificación que le asignó el crítico $r$ a la película $i$

In [2]:
# Obtiene una lista con los ítems en común entre dos usuarios
def shared_items(data, user_a, user_b):
    return [item for item in data[user_a] if item in data[user_b]]

In [3]:
shared_items(critics, "Gene Seymour", "Lisa Rose")

['Lady in the Water',
 'Snakes on a Plane',
 'Just My Luck',
 'Superman Returns',
 'You, Me and Dupree',
 'The Night Listener']

In [8]:
from math import sqrt

In [9]:
# Calcula la distancia euclideana entre dos usuarios en la base de datos
def euclidean_distance(data, user_a, user_b):
    return sqrt(sum([pow(data[user_a][item] - data[user_b][item], 2)
                     for item in shared_items(data, user_a, user_b)]))

In [10]:
euclidean_distance(critics, "Gene Seymour", "Lisa Rose")

2.3979157616563596

In [11]:
# Calcula similaridad = 1/(1+distancia)
def euclidean_similarity(data, user_a, user_b):
    return 1/(1+euclidean_distance(data, user_a, user_b))

In [12]:
euclidean_similarity(critics, "Gene Seymour", "Lisa Rose")

0.29429805508554946

#### Puntuación de la correlación de Pearson

Una manera mas sofisticada para determinar la similaridad entre los intereses de dos personas es usando el coeficiente de correlación de Pearson.

Considerando a dos críticos $a$ y $b$ podemos encontrar que tan similares son sus gustos si tomamos las películas que calificaron en común y las colocamos en un espacio bidimensional, en donde cada punto es $(c^{(a)}_i, c^{(b)}_i)$. Donde $c^{(r)}_i$ es la calificación que le asignó el crítico $r$ a la película $i$.

Se hace una regresión lineal con los puntos para obtener una recta con la que podemos determinar que tan cerca o lejos están las calificaciones de los críticos de ella.

In [24]:
def critics_ratings(data, user_a, user_b):
    shared = shared_items(data, user_a, user_b)
    c_a = [data[user_a][item] for item in shared]
    c_b = [data[user_b][item] for item in shared]
    n = len(shared)
    return c_a, c_b, n

In [25]:
c_a, c_b, n = critics_ratings(critics, "Gene Seymour", "Lisa Rose")
print "c_a = ", c_a
print "c_b = ", c_b
print "n   = ", n

c_a =  [3.0, 3.5, 1.5, 5.0, 3.5, 3.0]
c_b =  [2.5, 3.5, 3.0, 3.5, 2.5, 3.0]
n   =  6


Hay dos casos extremos que debemos considerar, primero el caso en el que dos críticos no tienen películas calificadas en común y luego el caso bizarro en el que la similaridad se indetermine (considere el caso de la similaridad de Pearson con "Luque" y "Moises".

En ambos casos se regresa 0 (lo que significa que no hay correlación entre estos dos críticos). Para el caso en el que se indetermina el resultado es por una división por cero, la cual puede ocurrir si se permiten las calificaciones de 0 o si los dos críticos tienen solo una película en común y uno de ellos la calificó con 1, es un caso peculiar que no sé como arreglar, así que lo dejaremos en que no hay correlación en este caso.

In [44]:
def pearson_similarity(data, user_a, user_b):
    c_a, c_b, n = critics_ratings(data, user_a, user_b)
    if n == 0: return 0
    sum_c_a = sum(c_a)
    sum_c_b = sum(c_b)
    sum_c_a_sq = sum(map(lambda x,y:x*y, c_a, c_a))
    sum_c_b_sq = sum(map(lambda x,y:x*y, c_b, c_b))
    sum_c_prod = sum(map(lambda x,y:x*y, c_a, c_b))
    numerator = sum_c_prod - (sum_c_a*sum_c_b/float(n))
    denominator = sqrt((sum_c_a_sq-pow(sum_c_a,2)/float(n))*
                       (sum_c_b_sq-pow(sum_c_b,2)/float(n)))
    if denominator == 0: return 0
    return numerator/denominator

In [47]:
pearson_similarity(critics, "Lolo", "Bobo")

1.0

#### Ordenando usuarios

Se implementa una función que a partir de un usuario, regresa una lista ordenada de otros usuarios (de mas similares a menos similares).

In [69]:
def top_matches(data, user, n=5, similarity=pearson_similarity):
    return sorted([(similarity(data, user, other), other)
                   for other in data if other != user])[-1:-n-1:-1]

In [71]:
top_matches(critics, "Toby", n=3)

[(0.9999999999997465, 'Christian'),
 (0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle')]

### Recomendando ítems

### Coincidiendo productos

### Construyendo un recomendador de link de del.icio.us

### Filtrado basado en ítems

### Usando el conjunto de datos de MovieLens